In [25]:
from src.loan.constant.trainingpipeline import SCHEMA_FILE_PATH
from sklearn.model_selection import train_test_split
import pandas as pd
import os, sys
from src.loan.exception import ModelException
from src.loan.logger import logging
from scipy.stats import ks_2samp
from src.loan.utils.main_utils import load_numpy_array_data, load_object

In [26]:
from src.loan.utils.main_utils import read_yaml_file, write_yaml_file

In [3]:
schema_config = read_yaml_file("D:/Machine Learning project/Loan/Loan Project/loan_prediction/config/schema.yaml")

In [4]:
import yaml

def read_yaml_file(filename):
    with open(filename, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [5]:
a = read_yaml_file("D:/Machine Learning project/Loan/Loan Project/loan_prediction/config/schema.yaml")

In [6]:
filename = "D:/Machine Learning project/Loan/Loan Project/loan_prediction/research"

In [7]:
report = {"not":"ok","oy":"jok"}

In [8]:
def write_yaml_file(filename: str, data, replace:bool = False) -> None:
    try:
        if replace:
            if os.path.exists(filename):
                os.remove(filename)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w") as file:
            return yaml.dump(data,file,default_flow_style=False)
    except Exception as e:
        raise ModelException(e,sys)

In [9]:
def writ_yaml_file(data, filename):
    with open(filename, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

In [10]:
from src.loan.constant.trainingpipeline import PIPELINE_NAME, ARTIFACT_DIR

In [29]:
a:str = 'data_validation'
b:str = "drift_report"
c:str = "report.yaml"

In [37]:
d = os.path.join(a,b,c)

In [31]:
os.makedirs(d,exist_ok=True)

In [32]:
def write_yaml_file(filename: str, data, replace:bool = False) -> None:
    try:
        if replace:
            if os.path.exists(filename):
                os.remove(filename)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w") as file:
            return yaml.dump(data,file,default_flow_style=False)
    except Exception as e:
        raise ModelException(e,sys)

In [38]:
report = {}

In [39]:
for col in range(3):
    report.update({col:{"p_value": float(col+1),
                                        "drift_status": col -1}})

In [40]:
print(report)

{0: {'p_value': 1.0, 'drift_status': -1}, 1: {'p_value': 2.0, 'drift_status': 0}, 2: {'p_value': 3.0, 'drift_status': 1}}


In [41]:
e = os.path.join(d)

In [42]:
write_yaml_file(filename=e,data=report)

In [10]:
from src.loan.constant.trainingpipeline import TRAIN_FILE_PATH, TEST_FILE_NAME
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers

In [40]:
def make_model(X_train, num_classes):
    
    
    # Entry block
    x = Dense(396,activation="relu",input_shape=(X_train.shape[1]))(X_train)
    x = Dense(198, activation="relu")(x)
    x = Dense(99, activation="relu")(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(8, activation="relu")(x)
    
    if num_classes == 2:
        units = 1
    else: 
        units = num_classes
    
    outputs = Dense(units, activation="sigmoid")(x)
    
    
    return keras.Model(X_train,outputs)

In [48]:
def build_model(X_train):
        try:
            logging.info("ANN Model building Start")
            model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),   # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(32, activation='relu'),   # Hidden layer with 32 neurons and ReLU activation
    tf.keras.layers.Dense(16, activation='relu'),   # Hidden layer with 16 neurons and ReLU activation
    tf.keras.layers.Dense(8, activation='relu'),    # Hidden layer with 8 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 neuron (for binary classification) and Sigmoid activation
])
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            logging.info("Model Building is Complete")
            return model
        except Exception as e:
            raise ModelException(e,sys)

In [49]:
build_model(X_train=X_train)

<Sequential name=sequential_2, built=True>

In [50]:
class ANNClassifier:
    def __init__(self,X_train,y_train):
        try:
            self.X_train = X_train
            self.y_train = y_train
            self.model = self._build_model
        except Exception as e:
            raise ModelException(e,sys)
        
    def _build_model(self):
        try:
            logging.info("ANN Model building Start")
            model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(self.X_train.shape[1],)),  # Input layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),   # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(32, activation='relu'),   # Hidden layer with 32 neurons and ReLU activation
    tf.keras.layers.Dense(16, activation='relu'),   # Hidden layer with 16 neurons and ReLU activation
    tf.keras.layers.Dense(8, activation='relu'),    # Hidden layer with 8 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 neuron (for binary classification) and Sigmoid activation
])
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            logging.info("Model Building is Complete")
            return model
        except Exception as e:
            raise ModelException(e,sys)


In [27]:
X_path = "D:/Machine Learning project/Loan/Loan Project/loan_prediction/artifact/data_transformation/transformed/train.npy"
y_path = "artifact/data_transformation/transformed/test.npy"

In [28]:
train = os.path.join(X_path)

In [29]:
train=load_numpy_array_data(X_path)

In [30]:
X_train, y_train =(
    train[:,:-1],
    train[:,1],
)

In [31]:
X_train.shape[1]

396

In [32]:
len(X_train[0])

396

In [51]:
model = ANNClassifier(X_train=X_train,y_train=y_train)

In [54]:
acc = model._build_model()

c:\Users\Nishant Borkar\anaconda3\envs\lven\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
acc.fit(X_train,y_train)

12429/12429 ━━━━━━━━━━━━━━━━━━━━ 14s 984us/step - accuracy: 0.9934 - loss: 0.0182
